# 3D Cylinder Demo
***
This is an example of a 3D reaction diffusion system.

- The domain is a cylinder
- Species A is created at the right end of the cylinder
- Species B is created at the left end of the cylinder
- Diffusion occurs throughout the volume of the cylinder
- A and B are destroyed on contact
- Deterministic and Stochastic simulations are run

This example allows us to check if reaction diffusion dynamics are accurate.
***
## Setup the Environment
***

In [ ]:
import spatialpy

***
## Create the 3D Cylinder Model
***

In [ ]:
def create_3D_cylinder():
    model = spatialpy.Model("3D_cylinder")
    
    # Define Domain Type IDs as constants of the Model
    model.MIDDLE = 'Middle'
    model.EDGE1 = 'Edge1'
    model.EDGE2 = 'Edge2'
    
    # Domain
    domain = spatialpy.Domain.read_stochss_domain('3D_cylinder.smdl')
    model.add_domain(domain)

    model.staticDomain = True

    # Variables
    A = spatialpy.Species(name="A", diffusion_coefficient=0.1, restrict_to=[model.MIDDLE, model.EDGE1])
    B = spatialpy.Species(name="B", diffusion_coefficient=0.1, restrict_to=[model.MIDDLE, model.EDGE2])
    model.add_species([A, B])

    # Parameters
    k_react = spatialpy.Parameter(name="k_react", expression="1")
    k_creat1 = spatialpy.Parameter(name="k_creat1", expression="100")
    k_creat2 = spatialpy.Parameter(name="k_creat2", expression="100")
    left = spatialpy.Parameter(name="left", expression="0.5092013833059308")
    right = spatialpy.Parameter(name="right", expression="0.505804729089437")
    model.add_parameter([k_react, k_creat1, k_creat2, left, right])

    # Reactions
    r1 = spatialpy.Reaction(
        name="r1", restrict_to=[model.EDGE1],
        reactants={}, products={'A': 1},
        propensity_function="k_creat1 / left * vol",
        ode_propensity_function="k_creat1"
    )
    r2 = spatialpy.Reaction(
        name="r2", restrict_to=[model.EDGE2],
        reactants={}, products={'B': 1},
        propensity_function="k_creat2 / right * vol",
        ode_propensity_function="k_creat2"
    )
    r3 = spatialpy.Reaction(
        name="r3", rate="k_react",
        reactants={'A': 1, 'B': 1}, products={}
    )
    model.add_reaction([r1, r2, r3])

    # Timespan
#     tspan = spatialpy.TimeSpan.arange(1, t=500, timestep_size=1e-3)
    tspan = spatialpy.TimeSpan.arange(1, t=500, timestep_size=1)
    model.timespan(tspan)
    return model

### Instantiate the Model

In [ ]:
model = create_3D_cylinder()

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    kwargs = {
        # "number_of_trajectories":1,
        # "seed":None,
    }
    return kwargs

***
## Run the Simulation
***

In [ ]:
kwargs = configure_simulation()
results = model.run(**kwargs)

***
## Visualization
***

In [ ]:
results.plot_species('A', animated=True, width=None, height=None)